In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
data = pd.read_csv('./datasets/data.csv', encoding='CP949')
target = pd.read_csv('./datasets/OECD신용등급.csv', encoding='CP949')
target.drop('신용등급_1', axis=1, inplace=True)

In [3]:
datasets = pd.merge(data, target, on=('국가별', '시점'), how='left')
datasets.columns = ['국가별', '시점', '인당_국민총소득', 'GDP_성장률', '디플레이터', '수출', '수입',
       '무역의존도_수출', '무역의존도_수입', '외환보유액', '부채비율', '국민부담률', '평균근로자세금', '경제활동참가율',
       '고용률', '실업률', '신용등급_1']
datasets['수입'] = datasets['수입'].astype(float)
datasets['무역의존도_수입'] = datasets['무역의존도_수입'].astype(float)
datasets.dropna(inplace=True)
datasets.reset_index(inplace=True)
datasets.drop(['국가별', 'index'], axis=1, inplace=True)
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시점        116 non-null    int64  
 1   인당_국민총소득  116 non-null    float64
 2   GDP_성장률   116 non-null    float64
 3   디플레이터     116 non-null    float64
 4   수출        116 non-null    float64
 5   수입        116 non-null    float64
 6   무역의존도_수출  116 non-null    float64
 7   무역의존도_수입  116 non-null    float64
 8   외환보유액     116 non-null    int64  
 9   부채비율      116 non-null    float64
 10  국민부담률     116 non-null    float64
 11  평균근로자세금   116 non-null    float64
 12  경제활동참가율   116 non-null    float64
 13  고용률       116 non-null    float64
 14  실업률       116 non-null    float64
 15  신용등급_1    116 non-null    float64
dtypes: float64(14), int64(2)
memory usage: 14.6 KB


In [4]:
lb = LabelEncoder()
datasets['시점'] = lb.fit_transform(datasets['시점'])
datasets.describe()

,시점,인당_국민총소득,GDP_성장률,디플레이터,수출,수입,무역의존도_수출,무역의존도_수입,외환보유액,부채비율,국민부담률,평균근로자세금,경제활동참가율,고용률,실업률,신용등급_1
count,116.000000,116.000000,116.000000,116.000000,1.160000e+02,1.160000e+02,116.000000,116.000000,1.160000e+02,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,1.689655,33475.815517,3.329310,95.595690,2.217822e+05,2.229416e+05,37.006552,37.020345,8.167653e+04,73.539026,34.313793,38.507759,60.153448,65.972414,8.257759,16.979940
std,1.058414,21025.028449,3.148085,13.460676,2.805340e+05,3.268235e+05,22.166097,21.632006,1.823440e+05,41.213343,6.950307,8.705592,5.940245,7.047338,4.170646,3.344374
min,0.000000,4070.600000,-5.500000,58.200000,3.092000e+03,3.919000e+03,7.440000,2.240000,2.790000e+02,6.788000,11.400000,14.700000,48.100000,46.300000,2.600000,3.900000
25%,1.000000,16716.900000,1.800000,87.700000,4.842350e+04,4.620275e+04,20.292500,22.902500,9.268750e+03,45.829500,30.500000,32.825000,57.025000,60.650000,5.100000,14.500000
50%,2.000000,30761.750000,2.950000,100.000000,1.204630e+05,8.910150e+04,31.785000,29.410000,3.073500e+04,63.252000,33.100000,38.800000,59.650000,66.100000,7.450000,18.000000
75%,3.000000,44388.900000,4.200000,103.700000,3.073575e+05,3.133178e+05,51.107500,50.787500,7.170775e+04,101.067500,39.250000,43.775000,63.125000,71.575000,9.925000,20.000000
max,3.000000,110886.000000,25.200000,143.100000,1.503400e+06,2.248800e+06,146.180000,137.700000,1.233153e+06,234.073000,48.000000,57.100000,81.400000,83.800000,24.900000,20.000000


In [5]:
datasets.iloc[:,15] = (round(datasets.iloc[:,15]).astype(int)).astype(str)

In [6]:
np.unique(datasets.iloc[:,15])

array(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '4', '9'], dtype=object)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(datasets.iloc[:,:15], datasets.iloc[:,15], test_size=0.2, random_state=42)

In [8]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_train_over, y_train_over = smote.fit_resample(x_train, y_train)
# print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", x_train.shape, y_train.shape)
# print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', X_train_over.shape, y_train_over.shape)
# print('SMOTE 적용 후 값의 분포 :\n',pd.Series(y_train_over).value_counts() )

In [9]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [10]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
acc = accuracy_score(pred, y_test)
# prec = precision_score(pred, y_test)
# reca = recall_score(pred, y_test)
# f1 = f1_score(pred, y_test)
#auc = roc_auc_score(pred, y_test)

print('정확도 : {:.2f}'.format(acc*100))
# print('정밀도 : {:.2f}'.format(prec*100))
# print('재현율 : {:.2f}'.format(reca*100))
# print('f1 score : {:.2f}'.format(f1*100))
#print('AUC : {:.2f}'.format(auc*100))

정확도 : 50.00


In [11]:
params = { "penalty" : ["l2", "l1"], "C" : [0.01, 0.1, 1, 5, 10] } 
lr = LogisticRegression() 
gs = GridSearchCV(lr, param_grid=params, cv=5, n_jobs=-1, scoring="accuracy") 
gs.fit(x_train, y_train) 
print("best param : {}".format(gs.best_params_)) 
print("best acc : {}".format(gs.best_score_))

d:\Python310\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


best param : {'C': 5, 'penalty': 'l2'}
best acc : 0.5538011695906432


d:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "d:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Solver lbfgs supports only 'l2' or 'none

In [15]:
pred = gs.predict(x_test)
accuracy_score(pred, y_test)

0.4166666666666667